In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/home/beans/bespoke')

from constants import *
from imports import *

In [2]:
%%time
get_obs_per_sec()

CPU times: user 4.01 ms, sys: 0 ns, total: 4.01 ms
Wall time: 3.51 ms


(94.6, 6.26)

In [6]:
%%time
get_mins_since_slowest_runner_reported()

ValueError: max() arg is an empty sequence

In [36]:
for i in range(10_000):
    print(round(get_mins_since_slowest_runner_reported(), 2), get_obs_per_sec())
    time.sleep(60*1)

/home/beans/bespoke/constants.py:435: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  propref_aux_target['ixx'] = list(range(len(propref_aux_target))) # dumb self ix so can grab sigmoid from within this smaller df


0.29 (22.66, 7.28)
1.29 (71.05, 5.42)
2.29 (76.38, 5.33)
3.12 (78.25, 5.33)
3.28 (86.39, 5.42)
2.59 (88.98, 6.0)
3.06 (87.8, 6.0)
3.67 (78.55, 5.32)
3.87 (76.37, 5.32)
2.48 (75.06, 5.32)
2.53 (93.59, 5.39)
2.91 (97.75, 5.39)
2.66 (101.82, 6.57)
3.36 (101.74, 6.57)
3.43 (94.62, 6.15)
2.81 (86.28, 5.8)
2.89 (88.15, 5.8)
2.45 (95.72, 6.2)
3.09 (96.13, 6.2)
3.37 (88.82, 5.55)
3.62 (88.53, 5.55)
2.95 (81.79, 5.55)
1.97 (87.13, 5.72)
1.98 (100.1, 6.84)
2.98 (104.8, 6.84)
3.21 (100.33, 5.5)
3.41 (91.96, 5.5)
2.1 (83.09, 5.49)
3.06 (88.55, 5.49)
3.32 (93.93, 5.49)
3.62 (92.16, 5.49)
2.99 (85.56, 5.76)
3.2 (87.93, 5.76)
2.75 (91.02, 5.98)
2.73 (88.8, 5.98)
2.64 (89.97, 5.98)
2.35 (92.65, 6.55)
3.17 (96.87, 6.55)
3.21 (95.37, 6.13)
3.06 (87.97, 6.13)
3.28 (84.43, 5.87)
2.52 (85.19, 5.85)
3.05 (91.01, 5.85)
2.63 (91.57, 5.85)
3.1 (97.93, 6.23)
2.63 (92.17, 6.23)
2.4 (95.51, 6.34)
2.74 (94.74, 6.34)
2.74 (95.17, 6.28)
3.45 (90.42, 6.18)
2.85 (85.78, 5.87)
3.47 (85.59, 5.87)
3.86 (91.36, 5.87)
2.87

KeyboardInterrupt: 

In [2]:
# Shut down trainer
set_trainer_should_stop(True)

In [27]:
# Shut down loader
#set_loader_should_stop(True)

In [3]:
# Shut down datagen
set_should_stop(True)

In [3]:
# set lr
set_lr(1e-4)

In [37]:
dataloader_root = f"{BLENDER_MEMBANK_ROOT}/dataloader_06"
current_run = np.load(f"{dataloader_root}/run_counter.npy")[0]
current_run

40